In [ ]:
import os, sys
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd

import ROOT
import root_numpy as rn
from larocv import larocv

rse    = ['run','subrun','event']
rsev   = ['run','subrun','event','vtxid']
rserv  = ['run','subrun','event','roid','vtxid']

def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [ ]:
name='inclusive_elec'
LLCUT=-14.625
all_df = pd.read_pickle(os.path.join(".","ll_bin","{}_all.pkl".format(name)))
LL_df = pd.read_pickle(os.path.join(".","ll_bin","{}_post_LL.pkl".format(name)))


if name == 'inclusive_elec':
    FILE = "/data/vgenty/tufts/inclusive_elec/ll_inter_ana.root"
if name == 'inclusive_muon':
    FILE = "/data/vgenty/tufts/inclusive_muon/comb_ana_ll.root"

mcinfo_df = pd.DataFrame(rn.root2array(FILE,treename='MCINFO'))
mcinfo_df = mcinfo_df.set_index(rse)

In [ ]:
LL_df.query("LL>@LLCUT").index.size

In [ ]:
nu_group = all_df.groupby(rse).nth(0)
LL_group = LL_df.set_index(rse)

res_comb    = pd.concat([nu_group,mcinfo_df],axis=1,join_axes=[nu_group.index])
res_comb_LL = pd.concat([LL_group,mcinfo_df],axis=1,join_axes=[LL_group.index])

In [ ]:
matplotlib.rcParams['font.size'] = 20
matplotlib.rcParams['font.family'] = 'serif'

fig,ax = plt.subplots(figsize=(10,6))
bins=np.arange(0,5000+100,100)

data0=res_comb.mcinfoEnergy.values*1000.0
data1=res_comb.query("num_croi>0").mcinfoEnergy.values*1000.0
data2=res_comb.query("num_croi>0 & good_croi_ctr>0").mcinfoEnergy.values*1000.0
data3=res_comb.query("num_croi>0 & good_croi_ctr>0 & min_vtx_dist < 5").mcinfoEnergy.values*1000.0
data4=res_comb_LL.query("num_croi>0 & good_croi_ctr>0 & scedr < 5 ").mcinfoEnergy.values*1000.0
data5=res_comb_LL.query("num_croi>0 & good_croi_ctr>0 & scedr < 5 & LL>@LLCUT").mcinfoEnergy.values*1000.0

weights0 = [1/float(data0.size)] * data0.size
weights1 = [1/float(data0.size)] * data1.size
weights2 = [1/float(data0.size)] * data2.size
weights3 = [1/float(data0.size)] * data3.size
# weights4 = [1/float(data0.size)] * data4.size
weights5 = [1/float(data0.size)] * data5.size

ax.hist(data0,weights=weights0,bins=bins,histtype='stepfilled',alpha=0.1,color='red')
ax.hist(data1,weights=weights1,bins=bins,histtype='stepfilled',alpha=0.1,color='green')
ax.hist(data2,weights=weights2,bins=bins,histtype='stepfilled',alpha=0.1,color='blue')
ax.hist(data3,weights=weights3,bins=bins,histtype='stepfilled',alpha=0.1,color='magenta')
# ax.hist(data4,weights=weights4,bins=bins,histtype='stepfilled',alpha=0.1,color='yellow')
ax.hist(data5,weights=weights5,bins=bins,histtype='stepfilled',alpha=0.1,color='cyan')

ax.hist(data0,weights=weights0,bins=bins,histtype='step',alpha=1.0,label='All Events',lw=3,color='red')
ax.hist(data1,weights=weights1,bins=bins,histtype='step',alpha=1.0,label='>= 1 cROI',lw=3,color='green')
ax.hist(data2,weights=weights2,bins=bins,histtype='step',alpha=1.0,label='>= 1 good cROI',lw=3,color='blue')
ax.hist(data3,weights=weights3,bins=bins,histtype='step',alpha=1.0,label='Good Vertex',lw=3,color='magenta')
# ax.hist(data4,weights=weights4,bins=bins,histtype='step',alpha=1.0,label='... & Nue Assumption',lw=3,color='yellow')
ax.hist(data5,weights=weights5,bins=bins,histtype='step',alpha=1.0,label='... & Pass Nue Selection',lw=3,color='cyan')

legend = ax.legend(fontsize=18)
ax.set_xlabel("True Neutrino Energy [MeV]",fontweight='bold')
ax.set_ylabel("Event Fraction",fontweight='bold')
ax.grid()
ax.set_ylim(0,0.08)

ax.set_ylim(0,0.061)
legend.remove()
ax.set_xlim(200,800)
plt.savefig("inclusive_all_E_%s.pdf" % name)
plt.show()


In [ ]:
code_to_name = {}
code_to_name[1000] = "NuanceOffset"
code_to_name[1001] = "CCQE"
code_to_name[1003] = "ResCCNuProtonPiPlus"
code_to_name[1004] = "ResCCNuNeutronPi0"
code_to_name[1005] = "ResCCNuNeutronPiPlus"
code_to_name[1091] = "CCDIS"
code_to_name[1097] = "CCCOHPi"

ix=-1
for df in [res_comb,
           res_comb.query("num_croi>0"),
           res_comb.query("num_croi>0 & good_croi_ctr>0"),
           res_comb.query("num_croi>0 & good_croi_ctr>0 & min_vtx_dist < 5"),
           res_comb_LL.query("num_croi>0 & good_croi_ctr>0 & scedr < 5 "),
           res_comb_LL.query("num_croi>0 & good_croi_ctr>0 & scedr < 5 & LL>@LLCUT")]:
    print df.index.size
    ix+=1
    if ix !=5: 
        continue
        
    fig,ax = plt.subplots(figsize=(10,6))
    data_v = []
    inter_v = []
    ratio_v = []
    weights_v = []
    for name, group in df.groupby("mcinfoInteractionType"):
        data = group.mcinfoEnergy.values * 1000.0
        weights = [1/float(res_comb.index.size)] * data.size
        ratio = float(data.size) / float(res_comb.index.size)
        label = "%s (%.02f%%)" % (code_to_name[name],ratio)
        ratio_v.append(ratio)
        inter_v.append(label)
        data_v.append(data)
        weights_v.append(weights)

    ax.hist(data_v,weights=weights_v,bins=bins,stacked=True,histtype='stepfilled',alpha=1.0,label=inter_v)
    ax.hist([],histtype='step',color='white',alpha=0.0,label='Total: %.02f%%' % np.array(ratio_v).sum())
    
    legend=ax.legend(fontsize=15)
    ax.set_xlabel("True Neutrino Energy [MeV]",fontweight='bold')
    ax.set_ylabel("Event Fraction",fontweight='bold')
    ax.set_ylim(0,0.08)
    ax.grid()

    if ix == 5:
        ax.set_ylim(0,0.015)     
        legend.remove()
        ax.set_xlim(200,800)
        
    plt.savefig("inclusive_all_E_breakdown_%s_%d.pdf" % (ix,name))
    plt.show()


In [ ]:
fig,ax = plt.subplots(figsize=(10,6))
data_v = []
inter_v = []
weights_v = []
for name, group in res_comb_LL.query("scedr<5").groupby("mcinfoInteractionType"):
    data = group.mcinfoEnergy.values * 1000.0
    weights = [1/float(LL_group.index.size)] * data.size
    inter_v.append(name)
    data_v.append(data)
    weights_v.append(weights)

ax.hist(data_v,weights=weights_v,bins=bins,stacked=True,histtype='stepfilled',alpha=1.0,label=inter_v)

ax.legend(fontsize=18)
ax.set_xlabel("True Neutrino Energy [MeV]",fontweight='bold')
ax.set_ylabel("Event Fraction",fontweight='bold')
#ax.set_ylim(0,0.08)
ax.grid()
plt.show()


fig,ax = plt.subplots(figsize=(10,6))
data_v = []
inter_v = []
weights_v = []
for name, group in res_comb_LL.query("LL>@LLCUT & scedr < 5").groupby("mcinfoInteractionType"):
    data = group.mcinfoEnergy.values * 1000.0
    weights = [1/float(LL_group.index.size)] * data.size
    inter_v.append(name)
    data_v.append(data)
    weights_v.append(weights)

ax.hist(data_v,weights=weights_v,bins=bins,stacked=True,histtype='stepfilled',alpha=1.0,label=inter_v)

ax.legend(fontsize=18)
ax.set_xlabel("True Neutrino Energy [MeV]",fontweight='bold')
ax.set_ylabel("Event Fraction",fontweight='bold')
#ax.set_ylim(0,0.08)
ax.grid()
plt.show()